# Transformer using PyTorch

Replicating the paper "Attention Is All You Need" from scratch using PyTorch.

Available here: https://arxiv.org/abs/1706.03762

In [2]:
import torch
import torch.nn as nn

import math


## Building the Encoder blocks

### Input Embeddings
In NLP, words/tokens are represented as integers (token IDs), but since neural networks cannot work directly with numbers, these are first converted into continuous dense vectors via embeddings.

The embedding layer:
- Maps each token ID to a learnable vector of dimension `d_model`
- Allows the model to learn semantic meaning - similar words/tokens get similar embeddings during training

Refer to section 3.4 of the paper.

In [ ]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model) #maps token IDs to vectors

    def forward(self, x):
        """
        Args:
            x: Tensor of token indices, shape (batch_size, seq_len)

        Returns:
            Embedded tensor, scaled, shape (batch_size, seq_len, d_model)
        
        """
        return self.embedding(x) * math.sqrt(self.d_model)  #Refer section 3.4 of the paper (This is primarily done to balance the scale of the embeddings and positional encodings, and helps stabilize training early on.)


### Positional Encoding
Transformers process tokens in parallel, with no built-in sense of order. But language is sequential:

"The cat sat on the mat" =/= "The mat cat on the sat"

Positional information is injected into the input to create awareness of the token position, using Positional Encoding. They have the same dimension `d_model` as the input embeddings.

Refer to section 3.5 of the paper.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        #Create matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        #Create a vector of shape (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #tensor contains positions 0,1,...,seq_len - 1
        #Using section 3.5 to write out position encodings
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -math.log(10000.0) / d_model)

        pe[:, 0::2] = torch.sin(position * div_term) #sine to even positions
        pe[:, 1::2] = torch.sin(position * div_term) #cosine to odd positions

        #Add batch dimension to make the shape (1, seq_len, d_model), so it can be added to x during forward pass
        #with shape (batch_size, seq_len, d_model)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe) #tensor gets saved with the model and moves with it to GPU/CPU

    def forward(self, x):
        x = x + (self.pe[:, :x.shape(1), :]).requires_grad_(False) #Add positional encodings to embeddings, requires_grad_() False means that this won't participate in backpropagation
        return self.dropout(x)
        

### Add + Norm Layer

- Layer Norm normalizes each sample’s hidden vector across dimensions, improving training dynamics and convergence.
- LayerNorm is preferred over BatchNorm because:
    1. It doesn't depend on batch statistics.
    2. Works well for variable-length sequences.

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, epsilon: float = 10**-6):
        super().__init__()
        self.epsilon = epsilon
        self.bias = nn.Parameter(torch.zeros(1)) #added (shift)
        self.alpha = nn.Parameter(torch.ones(1)) #multiplied (scale)

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        std = x.std(dim = -1, keepdim = True)

        return self.alpha * (x - mean) / (std + self.epsilon) + self.bias #From section


### Feed Forward layer

Discussed in section 3.3 of the paper, and equation 2

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff) #W1 and B1
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model) #W2 and B2

    def forward(self, x):
        return self.linear2(self.dropout(torch.relu(self.linear1(x)))) #(batch, seq_len, d_model) -> (batch, seq_len, d_ff) -> (batch, seq_len, d_model)

### Multi-Head Attention

Discussed in section 3.2.2 of the paper, this is the core innovation of the transformer model.
It is multiple scaled dot-product attention heads run in parallel, followed by a linear projection.

Refer to equation 1 for calculating attention scores, and then two equations in section 3.2.2 for multihead attention scores

Steps involved:
1. Project input `(Q, K, V)` into h subspaces using `W_q`, `W_k`, `W_v`.
2. Compute scaled dot-product attention for each head.
3. Concatenate all head outputs.
4. Pass through a final `W_o` projection layer.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, h, dropout): #h is the number of heads
        super().__init__()
        self.d_model = d_model #we have to ensure that d_model is divisible by h, d_model/h = d_k
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h

        self.w_q = nn.Linear(d_model, d_model) #W_q
        self.w_k = nn.Linear(d_model, d_model) #W_k
        self.w_v = nn.Linear(d_model, d_model) #W_v

        self.w_o = nn.Linear(d_model, d_model) #W_o
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1] #last dimension of Q,K,V

        # Calculate attention using Equation 1
        # (batch, h, seq_len, d_k) -> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k) #matrix multiplication of Q with K.transpose
        # Apply mask
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9) #values we don't want in the attention matrix will be replaced by a very low value
        
        # apply softmax
        attention_scores = attention_scores.softmax(dim = -1) #(batch, h, seq_len, seq_len)

        # apply dropout
        if dropout is not None:
            attention_scores = dropout(attention_scores)

        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)     # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
        key = self.w_k(k)       # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
        value = self.w_v(v)     # (batch, seq_len, d_model) -> (batch, seq_len, d_model)
 
        # Reshaping for MultiHead Attention from [batch, seq_len, d_model] to [batch, seq_len, h, d_k]
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2) #(batch, seq_len, d_model) -> (batch, seq_len, h, d_k).transpose -> (batch, seq_len, d_model) -> (batch, h, seq_len, d_k)

        x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)

        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k) #revert the transpose operation (batch, h, seq_len, d_k).transpose -> (batch, seq_len, d_model)
        
        #(batch, seq_len, d_model) -> (batch, seq_len, d_model)
        return self.w_o(x)

### Create skip connection (residual connection)